# 임베딩 및 가장 가까운 이웃 검색을 사용한 추천

추천은 웹 전반에 걸쳐 널리 사용되고 있습니다.

- '그 상품을 구매하셨나요? 비슷한 상품도 사용해 보세요.
- '그 책 재미있었나요? 비슷한 제목의 책을 읽어보세요.
- '찾고 있던 도움말 페이지가 없나요? 비슷한 페이지를 찾아보세요.

이 노트북은 임베딩을 사용해 추천할 유사한 항목을 찾는 방법을 보여줍니다. 특히, [AG의 뉴스 기사 말뭉치](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)를 데이터셋으로 사용합니다.

이 모델은 '어떤 기사가 주어졌을 때, 이 기사와 가장 유사한 다른 기사는 무엇인가'라는 질문에 답합니다.

----

Recommendations are widespread across the web.

- 'Bought that item? Try these similar items.'
- 'Enjoy that book? Try these similar titles.'
- 'Not the help page you were looking for? Try these similar pages.'

This notebook demonstrates how to use embeddings to find similar items to recommend. In particular, we use [AG's corpus of news articles](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) as our dataset.

Our model will answer the question: given an article, what other articles are most similar to it?

### 1. 임포트

먼저 나중에 필요한 패키지와 함수를 가져옵니다. 이 패키지들이 없다면 직접 설치해야 합니다. 터미널에서 `pip install {package_name}`을 실행하여 설치할 수 있습니다(예: `pip install pandas`).

---

First, let's import the packages and functions we'll need for later. If you don't have these, you'll need to install them. You can install them via your terminal by running `pip install {package_name}`, e.g. `pip install pandas`.

In [1]:
# imports
import pandas as pd
import pickle

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

# constants
EMBEDDING_MODEL = "text-embedding-ada-002"

### 2. 데이터 로드

다음으로 AG 뉴스 데이터를 로드하여 어떤 모습인지 확인해 보겠습니다.

---

Next, let's load the AG news data and see what it looks like.

In [2]:
# load data (full dataset available at http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)
dataset_path = "data/AG_news_samples.csv"
df = pd.read_csv(dataset_path)

# print dataframe
n_examples = 5
df.head(n_examples)

,title,description,label_int,label
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech


줄임표로 잘리지 않은 동일한 예시를 살펴보겠습니다.

---

Let's take a look at those same examples, but not truncated by ellipsis.

In [3]:
# print the title, description, and label of each example
for idx, row in df.head(n_examples).iterrows():
    print("")
    print(f"Title: {row['title']}")
    print(f"Description: {row['description']}")
    print(f"Label: {row['label']}")


Title: World Briefings
Description: BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.
Label: World

Title: Nvidia Puts a Firewall on a Motherboard (PC World)
Description: PC World - Upcoming chip set will include built-in security features for your PC.
Label: Sci/Tech

Title: Olympic joy in Greek, Chinese press
Description: Newspapers in Greece reflect a mixture of exhilaration that the Athens Olympics proved successful, and relief that they passed off without any major setback.
Label: Sports

Title: U2 Can iPod with Pictures
Description: SAN JOSE, Calif. -- Apple Computer (Quote, Chart) unveiled a batch of new iPods, iTunes software and promos designed to keep it atop the heap of digital music players.
Label: Sci/Tech

Title: The Dream Factory
Description: Any product, any shape, any size -- manufactured o

### 3. 임베딩을 저장하기 위한 캐시 구축

이 글에 대한 임베딩을 받기 전에 생성한 임베딩을 저장할 캐시를 설정해 보겠습니다. 일반적으로 나중에 다시 사용할 수 있도록 임베딩을 저장하는 것이 좋습니다. 저장하지 않으면 임베딩을 다시 계산할 때마다 비용을 다시 지불해야 합니다.

캐시는 `(text, model)`의 튜플을 플로트 목록인 임베딩에 매핑하는 딕셔너리입니다. 캐시는 파이썬 피클 파일로 저장됩니다.

----------

Before getting embeddings for these articles, let's set up a cache to save the embeddings we generate. In general, it's a good idea to save your embeddings so you can re-use them later. If you don't save them, you'll pay again each time you compute them again.

The cache is a dictionary that maps tuples of `(text, model)` to an embedding, which is a list of floats. The cache is saved as a Python pickle file.

In [4]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "data/recommendations_embeddings_cache.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

임베딩을 통해 작동하는지 확인해 보겠습니다.

---

Let's check that it works by getting an embedding.

In [5]:
# as an example, take the first description from the dataset
example_string = df["description"].values[0]
print(f"\nExample string: {example_string}")

# print the first 10 dimensions of the embedding
example_embedding = embedding_from_string(example_string)
print(f"\nExample embedding: {example_embedding[:10]}...")


Example string: BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.

Example embedding: [-0.01071077398955822, -0.022362446412444115, -0.00883542187511921, -0.0254171434789896, 0.031423427164554596, 0.010723662562668324, -0.016717055812478065, 0.004195375367999077, -0.008074969984591007, -0.02142154797911644]...


### 4. 임베딩을 기반으로 유사한 문서 추천

유사한 기사를 찾으려면 3단계 계획을 따르세요:
1. 모든 문서 설명의 유사성 임베딩을 가져옵니다.
2. 소스 제목과 다른 모든 기사 사이의 거리를 계산합니다.
3. 소스 제목과 가장 가까운 다른 기사를 인쇄합니다.

------

To find similar articles, let's follow a three-step plan:
1. Get the similarity embeddings of all the article descriptions
2. Calculate the distance between a source title and all other articles
3. Print out the other articles closest to the source title

In [6]:
def print_recommendations_from_strings(
    strings: list[str],
    index_of_source_string: int,
    k_nearest_neighbors: int = 1,
    model=EMBEDDING_MODEL,
) -> list[int]:
    """Print out the k nearest neighbors of a given string."""
    # 모든 문자열에 대한 임베딩 가져오기
    embeddings = [embedding_from_string(string, model=model) for string in strings]
    # 소스 문자열 임베딩 가져오기
    query_embedding = embeddings[index_of_source_string]
    # 소스 임베딩과 다른 임베딩 사이의 거리 가져오기(embeddings_utils.py의 함수)
    distances = distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine")
    # 가장 가까운 이웃의 인덱스 가져오기(embeddings_utils.py의 함수)
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

    # print out source string
    query_string = strings[index_of_source_string]
    print(f"Source string: {query_string}")
    # print out its k nearest neighbors
    k_counter = 0
    for i in indices_of_nearest_neighbors:
        # 시작 문자열과 동일한 문자열은 건너뜁니다.
        if query_string == strings[i]:
            continue
        # k개의 기사를 출력한 후 중지
        if k_counter >= k_nearest_neighbors:
            break
        k_counter += 1

        # print out the similar strings and their distances
        print(
            f"""
        --- Recommendation #{k_counter} (nearest neighbor {k_counter} of {k_nearest_neighbors}) ---
        String: {strings[i]}
        Distance: {distances[i]:0.3f}"""
        )

    return indices_of_nearest_neighbors

### 5. 추천 예시

토니 블레어에 관한 첫 번째 기사와 유사한 기사를 찾아보겠습니다.

----

Let's look for articles similar to first one, which was about Tony Blair.

In [13]:
article_descriptions = df["description"].tolist()

tony_blair_articles = print_recommendations_from_strings(
    strings=article_descriptions, # 기사 설명을 기반으로 유사성을 계산합니다.
    index_of_source_string=0, # 토니 블레어에 대한 첫 번째 기사와 유사한 기사를 살펴봅시다.
    k_nearest_neighbors=5, # 가장 유사한 기사 5개를 살펴봅시다.
)

Source string: BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.

        --- Recommendation #1 (nearest neighbor 1 of 5) ---
        String: THE re-election of British Prime Minister Tony Blair would be seen as an endorsement of the military action in Iraq, Prime Minister John Howard said today.
        Distance: 0.153

        --- Recommendation #2 (nearest neighbor 2 of 5) ---
        String: LONDON, England -- A US scientist is reported to have observed a surprising jump in the amount of carbon dioxide, the main greenhouse gas.
        Distance: 0.160

        --- Recommendation #3 (nearest neighbor 3 of 5) ---
        String: The anguish of hostage Kenneth Bigley in Iraq hangs over Prime Minister Tony Blair today as he faces the twin test of a local election and a debate by his Labour Party about the di

꽤 괜찮네요! 5개의 추천 항목 중 4개는 토니 블레어를 명시적으로 언급하고 있으며, 다섯 번째는 토니 블레어와 자주 연관될 수 있는 주제인 기후 변화에 관한 런던의 기사입니다.

---

Pretty good! 4 of the 5 recommendations explicitly mention Tony Blair and the fifth is an article from London about climate change, topics that might be often associated with Tony Blair.

보안이 강화된 NVIDIA의 새로운 칩셋에 대한 두 번째 예제 기사에서 추천 제품이 어떻게 작동하는지 확인해 보겠습니다.

---

Let's see how our recommender does on the second example article about NVIDIA's new chipset with more security.

In [14]:
chipset_security_articles = print_recommendations_from_strings(
    strings=article_descriptions, # 기사 설명에서 유사성을 기준으로 합니다.
    index_of_source_string=1, # 더 안전한 칩셋에 대한 두 번째 기사와 유사한 기사를 살펴봅시다.
    k_nearest_neighbors=5, # 가장 유사한 기사 5개를 살펴봅시다.
)

Source string: PC World - Upcoming chip set will include built-in security features for your PC.

        --- Recommendation #1 (nearest neighbor 1 of 5) ---
        String: PC World - Updated antivirus software for businesses adds intrusion prevention features.
        Distance: 0.112

        --- Recommendation #2 (nearest neighbor 2 of 5) ---
        String: PC World - The one-time World Class Product of the Year PDA gets a much-needed upgrade.
        Distance: 0.145

        --- Recommendation #3 (nearest neighbor 3 of 5) ---
        String: PC World - Send your video throughout your house--wirelessly--with new gateways and media adapters.
        Distance: 0.153

        --- Recommendation #4 (nearest neighbor 4 of 5) ---
        String: PC World - Symantec, McAfee hope raising virus-definition fees will move users to\  suites.
        Distance: 0.157

        --- Recommendation #5 (nearest neighbor 5 of 5) ---
        String: Gateway computers will be more widely available at Of

인쇄된 거리를 보면 1위 추천이 다른 모든 추천보다 훨씬 가깝다는 것을 알 수 있습니다(0.11 대 0.14 이상). 그리고 1위 추천은 시작 기사와 매우 유사해 보이는데, 컴퓨터 보안 강화에 관한 PC World의 또 다른 기사입니다. 꽤 괜찮네요! 

------

From the printed distances, you can see that the #1 recommendation is much closer than all the others (0.11 vs 0.14+). And the #1 recommendation looks very similar to the starting article - it's another article from PC World about increasing computer security. Pretty good! 

## Appendix: 보다 정교한 추천자에서 임베딩 사용

추천 시스템을 보다 정교하게 구축하는 방법은 항목 인기도나 사용자 클릭 데이터와 같은 수십 또는 수백 개의 신호를 수신하는 머신 러닝 모델을 훈련하는 것입니다. 이 시스템에서도 임베딩은 추천 시스템에 매우 유용한 신호가 될 수 있으며, 특히 아직 사용자 데이터가 없는 '콜드 스타트' 중인 품목(예: 아직 클릭이 없는 카탈로그에 추가된 새 제품)의 경우 더욱 그렇습니다.

---

A more sophisticated way to build a recommender system is to train a machine learning model that takes in tens or hundreds of signals, such as item popularity or user click data. Even in this system, embeddings can be a very useful signal into the recommender, especially for items that are being 'cold started' with no user data yet (e.g., a brand new product added to the catalog without any clicks yet).

## Appendix: 임베딩을 사용하여 유사한 문서 시각화하기

가장 가까운 이웃 추천자가 어떤 일을 하고 있는지 이해하기 위해 기사 임베딩을 시각화해 보겠습니다. 각 임베딩 벡터의 2048개 차원을 플로팅할 수는 없지만, [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) 또는 [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis)와 같은 기술을 사용하여 임베딩을 2차원 또는 3차원으로 압축하여 차트로 나타낼 수 있습니다.

가장 가까운 이웃을 시각화하기 전에 t-SNE를 사용하여 모든 기사 설명을 시각화해 보겠습니다. t-SNE는 결정론적이지 않으므로 실행할 때마다 결과가 달라질 수 있다는 점에 유의하세요.

-------

To get a sense of what our nearest neighbor recommender is doing, let's visualize the article embeddings. Although we can't plot the 2048 dimensions of each embedding vector, we can use techniques like [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) or [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) to compress the embeddings down into 2 or 3 dimensions, which we can chart.

Before visualizing the nearest neighbors, let's visualize all of the article descriptions using t-SNE. Note that t-SNE is not deterministic, meaning that results may vary from run to run.

In [ ]:
# 모든 문서 설명에 대한 임베딩 가져오기
embeddings = [embedding_from_string(string) for string in article_descriptions]
# t-SNE를 사용하여 2048차원 임베딩을 2차원으로 압축합니다.
tsne_components = tsne_components_from_embeddings(embeddings)
# 차트에 색을 입히기 위한 기사 레이블 가져오기
labels = df["label"].tolist()

chart_from_components(
    components=tsne_components,
    labels=labels,
    strings=article_descriptions,
    width=600,
    height=500,
    title="t-SNE components of article descriptions",
)

위의 차트에서 볼 수 있듯이 고도로 압축된 임베딩도 카테고리별로 문서 설명을 클러스터링하는 데 효과적입니다. 이 클러스터링은 레이블 자체에 대한 지식 없이도 수행된다는 점을 강조할 가치가 있습니다!

또한 가장 심각한 이상값을 자세히 살펴보면 잘못된 임베딩보다는 잘못된 라벨링이 원인인 경우가 많습니다. 예를 들어, 녹색 스포츠 클러스터에 있는 파란색 월드 포인트의 대부분은 스포츠 스토리로 보입니다.

---------

As you can see in the chart above, even the highly compressed embeddings do a good job of clustering article descriptions by category. And it's worth emphasizing: this clustering is done with no knowledge of the labels themselves!

Also, if you look closely at the most egregious outliers, they are often due to mislabeling rather than poor embedding. For example, the majority of the blue World points in the green Sports cluster appear to be Sports stories.

다음으로, 소스 문서인지, 가장 가까운 이웃 문서인지 또는 기타 문서인지에 따라 포인트의 색을 다시 지정해 보겠습니다.

-----

Next, let's recolor the points by whether they are a source article, its nearest neighbors, or other.

In [16]:
# 추천 글에 대한 레이블 만들기
def nearest_neighbor_labels(
    list_of_indices: list[int],
    k_nearest_neighbors: int = 5
) -> list[str]:
    """Return a list of labels to color the k nearest neighbors."""
    labels = ["Other" for _ in list_of_indices]
    source_index = list_of_indices[0]
    labels[source_index] = "Source"
    for i in range(k_nearest_neighbors):
        nearest_neighbor_index = list_of_indices[i + 1]
        labels[nearest_neighbor_index] = f"Nearest neighbor (top {k_nearest_neighbors})"
    return labels


tony_blair_labels = nearest_neighbor_labels(tony_blair_articles, k_nearest_neighbors=5)
chipset_security_labels = nearest_neighbor_labels(chipset_security_articles, k_nearest_neighbors=5
)

In [17]:
# 토니 블레어 기사의 가장 가까운 이웃을 보여주는 2D 차트
chart_from_components(
    components=tsne_components,
    labels=tony_blair_labels,
    strings=article_descriptions,
    width=600,
    height=500,
    title="Nearest neighbors of the Tony Blair article",
    category_orders={"label": ["Other", "Nearest neighbor (top 5)", "Source"]},
)

위의 2D 차트를 보면 토니 블레어에 대한 기사가 세계 뉴스 클러스터 내에서 어느 정도 서로 가깝다는 것을 알 수 있습니다. 흥미롭게도 가장 가까운 이웃 5개(빨간색)는 고차원 공간에서는 가장 가깝지만, 이 압축된 2D 공간에서는 가장 가까운 지점이 아닙니다. 임베딩을 2차원으로 압축하면 많은 정보가 손실되며, 2D 공간에서 가장 가까운 이웃은 전체 임베딩 공간의 이웃만큼 관련성이 높지 않은 것으로 보입니다.

----

Looking at the 2D chart above, we can see that the articles about Tony Blair are somewhat close together inside of the World news cluster. Interestingly, although the 5 nearest neighbors (red) were closest in high dimensional space, they are not the closest points in this compressed 2D space. Compressing the embeddings down to 2 dimensions discards much of their information, and the nearest neighbors in the 2D space don't seem to be as relevant as those in the full embedding space.

In [18]:
# 칩셋 보안 문서의 가장 가까운 이웃을 보여주는 2D 차트
chart_from_components(
    components=tsne_components,
    labels=chipset_security_labels,
    strings=article_descriptions,
    width=600,
    height=500,
    title="Nearest neighbors of the chipset security article",
    category_orders={"label": ["Other", "Nearest neighbor (top 5)", "Source"]},
)

칩셋 보안 예제의 경우, 전체 임베딩 공간에서 가장 가까운 4개의 이웃이 이 압축된 2D 시각화에서 가장 가까운 이웃으로 남아 있습니다. 다섯 번째는 전체 임베딩 공간에서는 더 가깝지만 더 먼 것으로 표시됩니다.

---

For the chipset security example, the 4 closest nearest neighbors in the full embedding space remain nearest neighbors in this compressed 2D visualization. The fifth is displayed as more distant, despite being closer in the full embedding space.

원한다면 `chart_from_components_3D` 함수를 사용하여 임베딩의 대화형 3D 플롯을 만들 수도 있습니다. (이렇게 하려면 `n_components=3`으로 t-SNE 컴포넌트를 다시 계산해야 합니다.)

---

Should you want to, you can also make an interactive 3D plot of the embeddings with the function `chart_from_components_3D`. (Doing so will require recomputing the t-SNE components with `n_components=3`.)